In [1]:
import glob

In [2]:
import os 

In [3]:
os.getcwd()

'/home/ec2-user/SageMaker'

In [4]:
no_map_scale_list = list()
map_scale_dict = dict()
map_scale_set = set()
# fp = r'/content/drive/MyDrive/GISR_code/images/Bathy_AOI_1_25m.tif'

for i in glob.glob('/home/ec2-user/SageMaker/data/*.LBL'):
    
    
    # print(i)
    
    
    
    # read the LBL file
    with open(i, 'r') as file:
        LBL_data = file.read()
#         print(LBL_data)
        if LBL_data.find('MAP_SCALE') != -1:
        
    #         print(LBL_data.find('MAP_SCALE'))
        
        
#         print(LBL_data.find('<METERS/PIXEL>'))
        
            map_scale = LBL_data[LBL_data.find('MAP_SCALE'):LBL_data.find('<METERS/PIXEL>')].replace(' ', '')
            # print(map_scale)
            map_scale_set.add(map_scale)
            if map_scale[10:] not in map_scale_dict:
                map_scale_dict[map_scale[10:]] = list()
            map_scale_dict[map_scale[10:]].append(i)

        else:
            
            no_map_scale_list.append(i)
            
#         # fp = 'data/NAC_DTM_A12LMAS.TIF'
#         # fp = r'Lunar_LROnac_Haworth_sfs-dem_1m_v3.tif'
#     img = rasterio.open(i)
    
    
#     show(img)
#     img_hr = img.read()[0]
#     print(img_hr.shape, img_hr.min(), img_hr.max())

In [5]:
no_map_scale_list, map_scale_set,map_scale_dict


([],
 {'MAP_SCALE=2.0',
  'MAP_SCALE=2.0000000000006',
  'MAP_SCALE=2.000000000006',
  'MAP_SCALE=2.999999999999',
  'MAP_SCALE=3.0',
  'MAP_SCALE=3.5',
  'MAP_SCALE=3.9999999999986',
  'MAP_SCALE=4.0',
  'MAP_SCALE=4.9999999999999',
  'MAP_SCALE=5.0',
  'MAP_SCALE=5.9999999999979',
  'MAP_SCALE=6.9999999999999',
  'MAP_SCALE=7.0'},
 {'2.0000000000006': ['/home/ec2-user/SageMaker/data/NAC_DTM_APOLLO16_1.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_SULPICIUS1.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_LICHTENBERG.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_GRUITHUISE2.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_HPONDS12.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_REINER4.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_ATLAS3.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_BRISBANEZ.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_FRESHMELT1.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_OPNHEIMRSE2.LBL',
   '/home/ec2-user/SageMaker/data/NAC_DTM_KINGCRATER4.LBL',
   

In [6]:
for i in map_scale_dict.values():
    print(len(i))

245
234
109
28
1
6
6
4
2
2
1
1
1


In [7]:
for i,j in map_scale_dict.items():
    print(i, len(j))

2.0000000000006 245
4.9999999999999 234
2.999999999999 109
3.9999999999986 28
5.0 1
2.0 6
5.9999999999979 6
4.0 4
3.0 2
6.9999999999999 2
2.000000000006 1
3.5 1
7.0 1


In [8]:
import torch
import torch.nn.functional as F
import rasterio
from rasterio.plot import show
import numpy as np
from tqdm import tqdm

In [9]:
def min_to_nan(img):
    
    import numpy as np
        
    if np.nanmin(img) == img.min():
        # min is approx. -3.4028227e+38
        
        img[img == img.min()] = np.nan
        
    return img
    

# convvert min to nan

In [10]:
# map_scale_dict['2.0000000000006']
not_downloaded_lst = list()
for i in tqdm(map_scale_dict['2.0000000000006']):
    print(i[30:-4])
    
    
    # fp = 'data/NAC_DTM_A12LMAS.TIF'
    # fp = r'Lunar_LROnac_Haworth_sfs-dem_1m_v3.tif'

    hr = rasterio.open(i)
    # show(hr)
    img_hr = hr.read()[0]

    # min 

    img_hr = min_to_nan(img_hr)
    # print(img_hr.shape, img_hr.min(), img_hr.max())
    print(img_hr.shape, img_hr.min(), img_hr.max(), np.nanmin(img_hr), np.nanmax(img_hr))

    break

  0%|          | 0/245 [00:00<?, ?it/s]

NAC_DTM_APOLLO16_1


  0%|          | 0/245 [00:00<?, ?it/s]

(14225, 2454) nan nan -168.28418 698.90796


In [11]:
min_to_nan(img_hr).min()

nan

In [12]:
# downscale images

In [13]:
import numpy as np
import os

def chop_and_save(array, A, output_dir, file_name):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Get the original dimensions
    N, M = array.shape
    
    # Calculate the dimensions after padding
    new_N = (N + A - 1) // A * A  # Next multiple of A
    new_M = (M + A - 1) // A * A
    
    # Create a new array with the padded size filled with NaN
    padded_array = np.full((new_N, new_M), np.nan)
    
    # Fill in the original array values into the padded array
    padded_array[:N, :M] = array
    
    # Chop the array into AxA blocks and save them
    block_id = 0
    for i in range(0, new_N, A):
        for j in range(0, new_M, A):
            block = padded_array[i:i+A, j:j+A]
            
            if not np.isnan(block).any():
            # Save the block to a file
                file_path = os.path.join(output_dir, "{}_block_{}.pt".format(file_name,block_id))
                torch.save(block, file_path)
                block_id += 1
            
            
    print(f"Saved {block_id} blocks in directory: {output_dir}")

# Example usage:
N, M, A = 5, 7, 3
array = np.arange(N * M).reshape(N, M)

output_directory = "/home/ec2-user/SageMaker/data/"
chop_and_save(array, A, output_directory, 'a')


Saved 2 blocks in directory: /home/ec2-user/SageMaker/data/


In [14]:
import numpy as np

def chop_and_fill(array, A):
    # Get the original dimensions
    N, M = array.shape
    
    # Calculate the dimensions after padding
    new_N = (N + A - 1) // A * A  # Next multiple of A
    new_M = (M + A - 1) // A * A
    
    # Create a new array with the padded size filled with NaN
    padded_array = np.full((new_N, new_M), np.nan)
    
    # Fill in the original array values into the padded array
    padded_array[:N, :M] = array
    
    # Chop the array into AxA blocks
    blocks = [
        padded_array[i:i+A, j:j+A]
        for i in range(0, new_N, A)
        for j in range(0, new_M, A)
    ]
    
    # Convert the list of blocks into a 4D array for convenience
    blocks_array = np.array(blocks).reshape(-1, A, A)
    
    return np.expand_dims(blocks_array,1)

# Example usage:
N, M, A = 5, 7, 3
array = np.arange(N * M).reshape(N, M)
result = chop_and_fill(array, A)

# Print the result
print("Original Array:\n", array)
print("\nChopped and Padded Blocks:")
for i, block in enumerate(result):
    print(f"Block {i}:\n{block}\n")


Original Array:
 [[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]]

Chopped and Padded Blocks:
Block 0:
[[[ 0.  1.  2.]
  [ 7.  8.  9.]
  [14. 15. 16.]]]

Block 1:
[[[ 3.  4.  5.]
  [10. 11. 12.]
  [17. 18. 19.]]]

Block 2:
[[[ 6. nan nan]
  [13. nan nan]
  [20. nan nan]]]

Block 3:
[[[21. 22. 23.]
  [28. 29. 30.]
  [nan nan nan]]]

Block 4:
[[[24. 25. 26.]
  [31. 32. 33.]
  [nan nan nan]]]

Block 5:
[[[27. nan nan]
  [34. nan nan]
  [nan nan nan]]]



In [15]:
M,N=256,256
images_tiles = [img_hr[x:x+M,y:y+N] for x in range(0,img_hr.shape[0],M) for y in range(0,img_hr.shape[1],N)]

In [16]:
len(images_tiles)

560

In [17]:
np.array(chop_and_fill(img_hr, 128)).shape

(2240, 1, 128, 128)

In [18]:
img_hr.copy().min() 

nan

In [19]:
map_scale = '2.0000000000006'
with open('DTMs_{}_urls.txt'.format(map_scale), 'w') as f:
    for i in map_scale_dict[map_scale]:
        site_name = i[30:-4]
        # print('https://pds.lroc.asu.edu/data/LRO-L-LROC-5-RDR-V1.0/LROLRC_2001/DATA/SDP/NAC_DTM/{}/{}.TIF'.format(site_name[8:],site_name))

        
        # f.write('https://pds.lroc.asu.edu/data/LRO-L-LROC-5-RDR-V1.0/LROLRC_2001/DATA/SDP/NAC_DTM/{}/{}.TIF\n'.format(site_name[8:],site_name))

In [20]:
scale_factor = 0.5

In [21]:
map_scale_dict['2.0000000000006']

['/home/ec2-user/SageMaker/data/NAC_DTM_APOLLO16_1.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_SULPICIUS1.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_LICHTENBERG.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_GRUITHUISE2.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_HPONDS12.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_REINER4.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_ATLAS3.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_BRISBANEZ.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_FRESHMELT1.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_OPNHEIMRSE2.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_KINGCRATER4.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_HSCARP4.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_WEIRDCRTR.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_HPONDS8.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_LICHTENBER4.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_LARMORQ3.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_AUSTRVOLC01.LBL',
 '/home/ec2-user/SageMaker/data/NAC_DTM_HORT

In [22]:
M

256

# Processing

In [ ]:


# map_scale_dict['2.0000000000006']
not_downloaded_lst = list()
for i in tqdm(map_scale_dict['2.0000000000006']):
    print(i[30:-4])
    
    
        # fp = 'data/NAC_DTM_A12LMAS.TIF'
        # fp = r'Lunar_LROnac_Haworth_sfs-dem_1m_v3.tif'
    try:
        hr = rasterio.open(i)
        # show(hr)
        img_hr = hr.read()[0]
        
        # min 
        img_hr = min_to_nan(img_hr)

        # print(img_hr)
        print(img_hr.shape, np.min(img_hr), np.max(img_hr), np.nanmin(img_hr), np.nanmax(img_hr))
        
#         chop_and_saev()
        
#         chopped_img_hr = np.array(chop_and_fill(img_hr, M))
#         tensor_img_hr = torch.tensor(chopped_img_hr)
        # tensor_img_lr = F.interpolate(tensor_img_hr, scale_factor = scale_factor, mode='bilinear')
        chop_and_save(img_hr, M, '/home/ec2-user/SageMaker/data_DEMs_no_nan/', i[30:-4])

        # torch.save(tensor_img_hr, '{}_hr.pt'.format(i[30:-4]))
        # tensor_img_lr.numpy()
        # torch.save(tensor_img_lr, '{}_lr.pt'.format(i[30:-4]))
        # print(tensor_img_hr.size(), tensor_img_hr.size())
        # print(tensor_img_lr.size(), tensor_img_lr.size())
        # break
        
    except:
        not_downloaded_lst.append(i)
    # break

  0%|          | 0/245 [00:00<?, ?it/s]

NAC_DTM_APOLLO16_1
(14225, 2454) nan nan -168.28418 698.90796


  0%|          | 1/245 [00:02<11:49,  2.91s/it]

Saved 432 blocks in directory: /home/ec2-user/SageMaker/data_DEMs_no_nan/
NAC_DTM_SULPICIUS1
NAC_DTM_LICHTENBERG
NAC_DTM_GRUITHUISE2
NAC_DTM_HPONDS12
NAC_DTM_REINER4
NAC_DTM_ATLAS3
NAC_DTM_BRISBANEZ
NAC_DTM_FRESHMELT1
NAC_DTM_OPNHEIMRSE2
NAC_DTM_KINGCRATER4
NAC_DTM_HSCARP4
NAC_DTM_WEIRDCRTR
NAC_DTM_HPONDS8
NAC_DTM_LICHTENBER4
NAC_DTM_LARMORQ3
NAC_DTM_AUSTRVOLC01
NAC_DTM_HORTENSIUS5
NAC_DTM_FECNDITATS2
NAC_DTM_GIORDNBRNO2
NAC_DTM_CHAPLYGIN2
NAC_DTM_TYCHOPK06
NAC_DTM_APOLLO16_2
NAC_DTM_TYCHOPK01
NAC_DTM_MRINGENII2
NAC_DTM_SULPICIUS2
NAC_DTM_APOLLO17_4
NAC_DTM_FECNDITATS3
(14366, 2329) nan nan -2321.731 -1940.038


 11%|█▏        | 28/245 [00:05<00:37,  5.78it/s]

...

NAC_DTM_RUMKERDOME7
NAC_DTM_PLANCKFLOOR
NAC_DTM_HPONDS1
NAC_DTM_HORTENSIUS7
NAC_DTM_GRUITHUISE9
NAC_DTM_NEWCRATER2
NAC_DTM_LUNA20_2
NAC_DTM_SPARIM2
NAC_DTM_INGHIRAMINW
NAC_DTM_SIMPELIUS02
NAC_DTM_APOLLO14
NAC_DTM_TRANQPIT1
NAC_DTM_DARWINC
NAC_DTM_BYRGIUSFLW1
(17621, 4442) nan nan -2095.1226 256.40878


 18%|█▊        | 44/245 [00:26<02:11,  1.53it/s]

Saved 744 blocks in directory: /home/ec2-user/SageMaker/data_DEMs_no_nan/
NAC_DTM_RANGER9
NAC_DTM_FEOKTISTOV
NAC_DTM_KARRER3
NAC_DTM_ANTONIADI
NAC_DTM_INGHIRAMI03
NAC_DTM_LUNA20
NAC_DTM_MRECRISIUM2
NAC_DTM_SPARIM4
NAC_DTM_FRSHCRATER13
NAC_DTM_RUMKERDOME4
NAC_DTM_PITISCUSW01
NAC_DTM_LICHTENBER9
NAC_DTM_SPARIM
NAC_DTM_GRUITHUISE1
NAC_DTM_BUFFON1
NAC_DTM_GIORDNBRNO4
NAC_DTM_KUGLERRIDGE
NAC_DTM_MOOREF2
NAC_DTM_HEVLIUSFLW1
NAC_DTM_JEANSU
NAC_DTM_CMPTNCRTR
NAC_DTM_BHABHAE01
NAC_DTM_KARRER2
NAC_DTM_LICHTENBER8
NAC_DTM_FRESHMELT2
NAC_DTM_GIORDNBRNO3
NAC_DTM_THEOPHILUS5
NAC_DTM_SURVEYOR7
NAC_DTM_ATLAS1
NAC_DTM_MAFICMOUND
NAC_DTM_VIRTANEN
NAC_DTM_DALEMBERT
NAC_DTM_LARMORQ1
NAC_DTM_COMPTONBELK
NAC_DTM_HPONDS5
NAC_DTM_BUFFON3
NAC_DTM_HPONDS6
NAC_DTM_REINER3
NAC_DTM_HSCARP2
NAC_DTM_APOLLO16_6
NAC_DTM_MRINGENII
(19383, 10311) nan nan -3650.4102 -2350.2207


 35%|███▍      | 85/245 [00:37<01:03,  2.54it/s]

NAC_DTM_MARIUS10
NAC_DTM_LASELMASIF1
NAC_DTM_LASELMASIF2
NAC_DTM_ARISTARCHU3
NAC_DTM_TYCHOPK07
NAC_DTM_ORIENTALE2
NAC_DTM_KINGCRATER3
NAC_DTM_HSCARP6
NAC_DTM_LARMORQ2
NAC_DTM_HPONDS2
NAC_DTM_APOLLO17_6
NAC_DTM_HSCARP3
NAC_DTM_MESSIER3
(14331, 2803) nan nan -3430.1677 -1259.7332


In [24]:
import numpy as np

# Create an example array
array = np.array([1, 2, np.nan, 4, 5])

# Check if the array contains any NaN
contains_nan = np.isnan(array).any()

print("Contains NaN:", contains_nan)


Contains NaN: True


# verify

In [13]:
for i in glob.glob(r'/home/ec2-user/SageMaker/data_DEMs_no_nan/*.pt'):
    
    array = torch.load(i)
                   
    print(i, np.min(array), np.max(array))
   

/home/ec2-user/SageMaker/data_DEMs_no_nan/NAC_DTM_HSCARP_block_1356.pt 599.0953369140625 646.1144409179688
/home/ec2-user/SageMaker/data_DEMs_no_nan/NAC_DTM_MRINGENII_block_371.pt -3503.313720703125 -3454.61962890625
/home/ec2-user/SageMaker/data_DEMs_no_nan/NAC_DTM_MRINGENII_block_682.pt -3198.4365234375 -3091.3818359375
/home/ec2-user/SageMaker/data_DEMs_no_nan/NAC_DTM_MESSIER3_block_134.pt -1584.28955078125 -1400.7822265625
/home/ec2-user/SageMaker/data_DEMs_no_nan/NAC_DTM_BYRGIUSFLW1_block_276.pt -1956.80419921875 -1835.7755126953125
/home/ec2-user/SageMaker/data_DEMs_no_nan/NAC_DTM_FRSHCRATER4_block_220.pt -2056.40380859375 -1964.6207275390625
/home/ec2-user/SageMaker/data_DEMs_no_nan/NAC_DTM_GIORDNBRNO5_block_463.pt -136.052978515625 -31.2724666595459
/home/ec2-user/SageMaker/data_DEMs_no_nan/NAC_DTM_APOLLO17_5_block_191.pt -2735.149658203125 -2701.157470703125
/home/ec2-user/SageMaker/data_DEMs_no_nan/NAC_DTM_VIRTANEN2_block_89.pt 996.4937133789062 1047.9041748046875
/home/ec2-u

EOFError: Ran out of input

In [18]:
import glob
import numpy as np

print(len(glob.glob('/home/ec2-user/SageMaker/data_DEMs_no_nan/train/*.pt')))
print(len(glob.glob('/home/ec2-user/SageMaker/data_DEMs_no_nan/test/*.pt')))
# print(len(np.random.choice(glob.glob('/home/ec2-user/SageMaker/data/train/*_hr.pt'),int(np.floor(len(glob.glob('/home/ec2-user/SageMaker/data/train/*_hr.pt'))*0.3)))))

1172
1674


In [19]:
np.random.seed(0)

In [20]:
np.random.choice(glob.glob('/home/ec2-user/SageMaker/data_DEMs_no_nan/train/*.pt'),500)

array(['/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_ARISTARCHU2_block_247.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_LICHTENBER13_block_318.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_FECNDITATS3_block_248.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_ARISTARCHU2_block_161.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_ARISTARCHU2_block_164.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_FECNDITATS3_block_70.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_FECNDITATS3_block_191.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_LICHTENBER13_block_200.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_FECNDITATS3_block_95.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_FECNDITATS3_block_173.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/train/NAC_DTM_FECNDITATS3_block_175.pt',

In [21]:
np.random.choice(glob.glob('/home/ec2-user/SageMaker/data_DEMs_no_nan/test/*.pt'),100)

array(['/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_CMPTNBELK2_block_86.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_CMPTNBELK2_block_486.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_CMPTNBELK2_block_192.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_CMPTNBELK2_block_382.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_CMPTNBELK2_block_489.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_CMPTNBELK2_block_26.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_CMPTNBELK2_block_359.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_CMPTNBELK2_block_3.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_MESSIER3_block_329.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_VIRTANEN2_block_204.pt',
       '/home/ec2-user/SageMaker/data_DEMs_no_nan/test/NAC_DTM_CMPTNBELK2_block_523.pt',
       '/home/ec2-user/SageM

In [2]:
import matplotlib.pyplot as plt
import torch
import numpy as np

In [9]:
t = torch.load('NAC_DTM_CHAPLYGIN2.pt')

In [10]:
t.shape

torch.Size([2944, 1, 64, 64])

In [14]:
np.random.choice(t.shape[0],1)

array([1884])

In [ ]:
lr = F.interpolate(hr, scale_factor = 0.5, mode='bilinear')
lr.size(), hr.size()

In [5]:
torch.load('/home/ec2-user/SageMaker/data/test/NAC_DTM_APOLLO16_1_hr.pt').shape

torch.Size([560, 1, 256, 256])

In [4]:
with open('/home/ec2-user/SageMaker/data/all_DTM_urls.txt','r') as f:
    new_lst = f.readlines()

In [9]:
np.random.seed(0)
small_lst = np.random.choice(new_lst,50)

with open('/home/ec2-user/SageMaker/data/small_set_DTM_urls.txt','w') as f:
    for i in small_lst:

        f.write(i)